In [1]:
from visual import feature_embedding
import sys
import torch
torch.backends.cudnn.benchmark = True
from utils import load_data
from model import CNNFeature,RNNFeature
from worker import seq_predict,basic_predict
import numpy as np
from reader import getVideoList
import pandas as pd
from utils import read_frames
from matplotlib import pyplot as plt
import json
def plot_acc(record,title):
    plt.plot(record['train_accuracy'],label='train')
    plt.plot(record['val_accuracy'],label='val')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title(title)
    plt.legend()
    plt.show()
def plot_loss(record,title):
    plt.plot(record['train_loss'],label='train')
    plt.plot(record['val_loss'],label='val')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title(title)
    plt.legend()
    plt.show()
def get_labels(path):
    labels = []
    with open(path,"r") as fp:
        for line in fp:
            labels.append(int(line.rstrip("\n")))
    return labels
def get_ann(path):
    indice = get_labels(path)
    indice = np.array(indice)
    one_hot = np.zeros((indice.size,indice.max()+1))
    one_hot[np.arange(indice.size),indice]=1
    one_hot = one_hot.transpose()
    return one_hot

Using TensorFlow backend.


In [2]:
input_path = 'hw4_data/TrimmedVideos/video/valid'
data = np.load('hw4_data/video_0_5_12/trimmed_val.npy')
labels = getVideoList('hw4_data/TrimmedVideos/label/gt_valid.csv')['Action_labels']
labels = np.array(labels).astype('int')

FileNotFoundError: [Errno 2] No such file or directory: 'hw4_data/video_0_5_12/trimmed_val.npy'

In [ ]:
model = CNNFeature().cuda()
model.load_state_dict(torch.load('model/answer_1_classifier.pth'),strict=False)
result = feature_embedding(model,data,labels,
                           "CNN-based features t-SNE result of vaildation data ",
                           basic_predict,get_max=False)

In [ ]:
data = np.load('hw4_data/video_0_5_12/trimmed_val_full.npy')
labels = getVideoList('hw4_data/TrimmedVideos/label/gt_valid.csv')['Action_labels']
labels = np.array(labels).astype('int')

In [ ]:
model = RNNFeature(hidden=512,num_layers=9,get_last=True).cuda()
model.load_state_dict(torch.load('model/answer_3_56.37_classifier.pth'),strict=False)

In [ ]:
result = feature_embedding(model,data,labels,
                           "RNN-based features t-SNE result of vaildation data ",
                           seq_predict,lengths=lengths,get_max=False)

In [ ]:
with open('model/answer_1_record.txt',"r") as fp:
    record_1 = json.load(fp)
plot_acc(record_1,'Accuracy per epoch of answer 1')
plot_loss(record_1,'Loss per epoch of answer 1')

In [ ]:
with open('model/answer_2_record.txt',"r") as fp:
    record_2 = json.load(fp)
plot_acc(record_2,'Accuracy per epoch of answer 2')
plot_loss(record_2,'Loss per epoch of answer 2')

In [ ]:
with open('model/answer_3_56.37_record.txt',"r") as fp:
    record_3 = json.load(fp)
plot_acc(record_3,'Accuracy per epoch of answer 3')
plot_loss(record_3,'Loss per epoch of answer 3')

In [ ]:
output = get_labels('output2/p1_valid.txt')
answer = np.array(getVideoList('hw4_data/TrimmedVideos/label/gt_valid.csv')['Action_labels']).astype('int')
sum(np.array(output)==np.array(answer))/len(output)

In [ ]:
output = get_labels('output2/p2_result.txt')
answer = np.array(getVideoList('hw4_data/TrimmedVideos/label/gt_valid.csv')['Action_labels']).astype('int')
sum(np.array(output)==np.array(answer))/len(output)

In [ ]:
outputs = []
answers = []
for name in ['OP01-R02-TurkeySandwich','OP01-R04-ContinentalBreakfast','OP01-R07-Pizza',
             'OP03-R04-ContinentalBreakfast','OP04-R04-ContinentalBreakfast',
             'OP05-R04-ContinentalBreakfast','OP06-R03-BaconAndEggs']:
    name += '.txt'
    output = get_labels('output2/'+name)
    answer = get_labels('hw4_data/FullLengthVideos/labels/valid/'+name)
    outputs += output
    answers += answer

In [ ]:
sum(np.array(outputs)==np.array(answers))/len(outputs)

In [ ]:
frames = read_frames('hw4_data/FullLengthVideos/videos/valid/OP04-R04-ContinentalBreakfast')

In [ ]:
for timestep in range(500,700,5):
    fig, ax = plt.subplots()
    ax = fig.add_axes([0, 0, 1, 1])
    ax.axis('off')
    ax.xaxis.set_label_position('top')
    plt.imshow(frames[timestep].transpose(1,2,0))
    plt.savefig("pic/timestep_{}.png".format(timestep))

In [ ]:
current_palette = ["red", "green", "blue", 
                   "purple","pink","gray",
                   "black","darkgreen","tan",
                   "deepskyblue","gold"]

In [ ]:
answer_one_hot_[:,200:210]

In [ ]:
answer_one_hot_ = get_ann('hw4_data/FullLengthVideos/labels/valid/OP04-R04-ContinentalBreakfast.txt')
predict_one_hot_ = get_ann('output2/OP04-R04-ContinentalBreakfast.txt')

In [ ]:
for step in range(0,500,100):
    first = 200 + step
    second = 300 + step
    answer_one_hot = answer_one_hot_[:,first:second]
    predict_one_hot = predict_one_hot_[:,first:second]
    fig, ax = plt.subplots()
    #ax = fig.add_axes([0, 0, 1, 1])
    #ax.axis('off')
    plt.stackplot(list(range(answer_one_hot.shape[1])),answer_one_hot,colors=current_palette)
    #plt.xlabel("Timestep of answer: {}~{}".format(first,second))
    plt.savefig("pic/answer_{}_to_{}.png".format(first,second))
    fig, ax = plt.subplots()
    ax = fig.add_axes([0, 0, 1, 1])
    #ax.axis('off')
    #ax.xaxis.set_label_position('top')
    plt.stackplot(list(range(predict_one_hot.shape[1])),predict_one_hot,colors=current_palette)
    #plt.xlabel("Timestep of prediction: {}~{}".format(first,second))
    plt.savefig("pic/prediction_{}_to_{}.png".format(first,second))